In [13]:
# pip install underthesea

In [14]:
import numpy as np
import pandas as pd
import underthesea
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [15]:
df_list = []
for i in range(1, 12):
    file_path = "../assets/nlp_class_dataset/G01.xlsx".replace("01", str(i).zfill(2))
    df_temp = pd.read_excel(file_path, engine='openpyxl')
    df_list.append(df_temp)

# Concatenate all dataframes (safe when df may not exist yet)
if df_list:
    df = pd.concat(df_list, ignore_index=True, sort=False)
else:
    df = pd.DataFrame()

df.head()

,id,title,content,topic
0,G01_001,Amorim: 'Man Utd có thể thắng nhiều hơn nếu đá...,Hệ thống 3-4-3 của Amorim gây tranh cãi từ khi...,Thể thao
1,G01_002,Madam Pang đến Đại sứ quán Việt Nam tại Thái L...,Madam Pang gặp Đại sứ đặc mệnh toàn quyền Việt...,Thể thao
2,G01_003,Ngày 'phán quyết' của bóng đá Malaysia,"FAMnộp đơn kháng cáongày 14/10. Trước đó, vào ...",Thể thao
3,G01_004,Các HLV người Tây Ban Nha thống trị châu Âu nh...,Pep Guardiola trao đổi với Bernardo Silva tron...,Thể thao
4,G01_005,Vì sao Djokovic chưa giải nghệ?,"Djokovic cứu bóng ở trận gặp Bergs, trên sân T...",Thể thao


In [ ]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 580 entries, 0 to 581
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       580 non-null    object
 1   title    580 non-null    object
 2   content  580 non-null    object
 3   topic    580 non-null    object
dtypes: object(4)
memory usage: 22.7+ KB


In [18]:
df

,id,title,content,topic
0,G01_001,Amorim: 'Man Utd có thể thắng nhiều hơn nếu đá...,Hệ thống 3-4-3 của Amorim gây tranh cãi từ khi...,Thể thao
1,G01_002,Madam Pang đến Đại sứ quán Việt Nam tại Thái L...,Madam Pang gặp Đại sứ đặc mệnh toàn quyền Việt...,Thể thao
2,G01_003,Ngày 'phán quyết' của bóng đá Malaysia,"FAMnộp đơn kháng cáongày 14/10. Trước đó, vào ...",Thể thao
3,G01_004,Các HLV người Tây Ban Nha thống trị châu Âu nh...,Pep Guardiola trao đổi với Bernardo Silva tron...,Thể thao
4,G01_005,Vì sao Djokovic chưa giải nghệ?,"Djokovic cứu bóng ở trận gặp Bergs, trên sân T...",Thể thao
...,...,...,...,...
577,G11_046,Thực hiện mạnh mẽ cuộc cách mạng tinh gọn bộ m...,"Trần Thường\nTheo TTXVN, Tiểu ban Nhân sự Đại ...",Quan hệ quốc tế
578,G11_047,Tư tưởng xuyên suốt của văn kiện Đại hội 14 là...,Thu Hằng\nXuân Ất Tỵ là thời khắc đánh dấu sự ...,Quan hệ quốc tế
579,G11_048,Bộ Công an thông tin chi tiết về 6 bị can vụ á...,"Thu Hằng\nChiều 31/12, tại cuộc họp thông tin ...",Quan hệ quốc tế
580,G11_049,Bộ máy Chính phủ sẽ tinh gọn thế nào sau sắp x...,Thu Hằng\nTrao đổi với VietNamNet về phương án...,Quan hệ quốc tế


In [19]:
def containDigit(iword):
    return any(ch.isdigit() for ch in iword)

def containUpperLetterButNotFirst(iword):
    if(iword.isupper()):
        return True
    for i in range(1, len(iword)):
        if(iword[i].isupper() and iword[i-1]!=' '):
            return False
    return True

def FeatureFunction1(w): # Chỉ lọc những từ không hợp lệ có độ dài từ 3 kí tự trở xuống VÀ có chứa số VÀ có chữ cái viết hoa không chuẩn (viết hoa không phải kí tự đầu)
    return len(w) > 3 and not containDigit(w) and containUpperLetterButNotFirst(w)

In [20]:
for i in range(len(df["title"])):
  lwt = underthesea.word_tokenize(df["title"].iloc[i].replace("'","").replace("\"",""))
  lwtl = [w for w in lwt if FeatureFunction1(w)]
  df.loc[i, "title"] = " | ".join(lwtl)

In [21]:
for i in range(len(df["content"])):
  lwc = underthesea.word_tokenize(df["content"].iloc[i].replace("'","").replace("\"",""))
  lwcl = [w for w in lwc if FeatureFunction1(w)]
  df.loc[i, "content"] = " | ".join(lwcl)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 582 entries, 0 to 144
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       580 non-null    object
 1   title    582 non-null    object
 2   content  582 non-null    object
 3   topic    580 non-null    object
dtypes: object(4)
memory usage: 38.9+ KB


In [23]:
display(df)

,id,title,content,topic
0,G01_001,Amorim | Man Utd | có thể | thắng | nhiều,Hệ thống | Amorim | tranh cãi | quyền | Old Tr...,Thể thao
1,G01_002,Madam | Pang | Đại sứ quán | Việt Nam | Thái L...,Madam | Pang | Đại sứ | đặc mệnh | toàn quyền ...,Thể thao
2,G01_003,Ngày | phán quyết | bóng đá | Malaysia,kháng | cáongày | Trước | ngày | Ủy ban | kỷ l...,Thể thao
3,G01_004,người | Tây Ban Nha thống trị | châu Âu | như ...,Guardiola | trao đổi | Bernardo Silva | trong ...,Thể thao
4,G01_005,Djokovic | chưa | giải nghệ,Djokovic | bóng | trận | Bergs | trên | Trung ...,Thể thao
...,...,...,...,...
579,G11_048,cơ quan | Đảng | Mặt trận | Tổ quốc | được | s...,Sáng | Bộ Chính trị | Ban Bí thư | Trung ương ...,Quan hệ quốc tế
580,G11_049,Bộ máy Chính phủ sẽ tinh gọn thế nào sau sắp x...,Văn phòng | Chính phủ | | phát | ý kiến | thủ ...,Quan hệ quốc tế
581,G11_050,"Các cơ quan Đảng, Mặt trận Tổ quốc sẽ được sắp...",Bộ trưởng | Ngoại giao | Lê Hoài Trung | | hôm...,Quan hệ quốc tế
138,NaN,Chính phủ | yêu cầu | nguyên nhân | dừng | kiể...,Văn phòng | Chính phủ | phát | ý kiến | thủ tư...,NaN


In [24]:
combined_df_by_category = df.groupby('topic').agg({
    'id': lambda x: ' | '.join(x),
    'title': lambda x: ' | '.join(x),
    'content': lambda x: ' | '.join(x)
}).reset_index()

display(combined_df_by_category)

,topic,id,title,content
0,Chính trị,G04_001 | G04_002 | G04_003 | G04_004 | G04_00...,Việt Nam | tuyến | đường sắt | tốc độ | tháng ...,thủ tướng | Trần Hồng Hà | phê duyệt | điều ch...
1,Du lịch,G09_001 | G09_002 | G09_003 | G09_004 | G09_00...,sông | Hương hút | khách | hóng | ngắm | hoàng...,được | chính quyền | chỉnh trang | chạy | sông...
2,Giáo dục,G06_001 | G06_002 | G06_003 | G06_004 | G06_00...,GD&ĐT | yêu cầu | siết | chặt | thêm | trái ph...,địa phương | phải | rà soát | xử lý | nghiêm |...
3,Giải trí,G02_001 | G02_002 | G02_003 | G02_004 | G02_00...,Diễn viên | Đình Tú | Ngọc Huyền | cưới | Dima...,Diễn viên | Đình Tú | tuổi | Ngọc Huyền | tuổi...
4,Khoa học và công nghệ,G05_001 | G05_002 | G05_003 | G05_004 | G05_00...,Tâm lý | không | theo | trào lưu ảnh | Tranh l...,thấy | bạn bè | chia sẻ | Hoài Anh | không | c...
5,Kinh doanh,G03_001 | G03_003 | G03_004 | G03_007 | G03_00...,tăng | mục tiêu | cuối | tăng trưởng | trên | ...,họp báo | sáng | Thống kê | Tài chính | biết |...
6,Quan hệ quốc tế,G11_001 | G11_002 | G11_003 | G11_004 | G11_00...,Việt Nam | nâng cấp | quan hệ | Đối tác | Chiế...,Thủ tướng | Keir | Starmer chiều | chủ trì | h...
7,Sức khoẻ - y tế,G08_001 | G08_002 | G08_003 | G08_004 | G08_00...,sơ sinh | bỏ rơi | đường | được | như thế nào ...,sơ sinh | bỏ rơi | được | bước | người dân | p...
8,Thể thao,G01_001 | G01_002 | G01_003 | G01_004 | G01_00...,Amorim | Man Utd | có thể | thắng | nhiều | Ma...,Hệ thống | Amorim | tranh cãi | quyền | Old Tr...
9,Xã hội - Đời sống,G10_001 | G10_002 | G10_003 | G10_004 | G10_00...,Học sinh | áp lực | điểm số | Cư dân | mạng | ...,Nhiều | học sinh | chia sẻ | stress | kỳ vọng ...


1. Tạo một bảng dữ liệu trong đó, mỗi từ là một feature, và giá trị của nó là số lần xuất hiện của nó. Có thể cộng cả content+title lại. Tiếp đến feed bảng dữ liệu này vào 3 mô hình học máy của scikit (Logistic, SVM, NaiveBayes)

2. Làm tương tự ở 1, nhưng bảng dữ liệu với số thuộc tính giảm sau khi loại các từ xuất hiện chung trong nhiều nhóm (có thể sử dụng set kết hợp với intersect - phép giao). Bảng features này có kích thước nhỏ hơn nhiều so với bảng 1.

3. Không tạo bảng features như 1 và 2 mà sử dụng kĩ thuật BoW (trong scikit, sử dụng CountVectorize)

4. Không tạo bảng feature như 1 và 2 mà sử dụng tfidf (TfidfVectorizer của scikit)

***Câu 1: xử lý thông thường, dùng tất cả các word hợp lệ (không lấy số, các từ có độ dài từ bé hơn hoặc bằng 3 kí tự)***

In [25]:
for i in range(len(df["content"])):
  df.loc[i, "text"] = str(df["content"][i] + " | " + df['title'][i])

combined_df=df.drop(labels=["id", "title", "content"], axis=1)

In [26]:
display(combined_df)

,topic,text
0,Thể thao,Hệ thống | Amorim | tranh cãi | quyền | Old Tr...
1,Thể thao,Madam | Pang | Đại sứ | đặc mệnh | toàn quyền ...
2,Thể thao,kháng | cáongày | Trước | ngày | Ủy ban | kỷ l...
3,Thể thao,Guardiola | trao đổi | Bernardo Silva | trong ...
4,Thể thao,Djokovic | bóng | trận | Bergs | trên | Trung ...
...,...,...
579,Quan hệ quốc tế,Sáng | Bộ Chính trị | Ban Bí thư | Trung ương ...
580,Quan hệ quốc tế,Văn phòng | Chính phủ | | phát | ý kiến | thủ ...
581,Quan hệ quốc tế,Bộ trưởng | Ngoại giao | Lê Hoài Trung | | hôm...
138,NaN,Văn phòng | Chính phủ | phát | ý kiến | thủ tư...


In [27]:
mega_docs = ""
for i in range(len(combined_df)):
  mega_docs += combined_df["text"].iloc[i]

vocab = mega_docs.split(" | ");
vocab_set = set(vocab)
print(vocab_set)

{'ubnd', 'thán phục', 'Quốc hội', 'Chỉ huy', 'hiếm trải', 'Gelex', 'Reuters Mối', 'Hồ Bắc', 'nhạc trưởng', 'khuya', 'Thịt', 'Bạo lực', 'Bảo Thắng', 'hoa mai', 'ngoại ngữ', 'liên tỉnh', 'tất bật', 'giật tít', 'Sora', 'nắng gió', 'Đơn giản', 'ngày sau', 'đánh đầu', 'ngậm', 'khinh thường', 'bánh bao', 'Vườn sầu', 'ban chấp hành', 'bận rộn', 'buốt', 'chàng', 'mặc định', 'thương binh', 'ứng viên', 'chứa đồ', 'Thế chiến', 'Cầu Kiệu', 'thần tượng', 'lặng lẽ', 'Guardian', 'dich', 'Hồ Đắc Minh Quân', 'đặt biệt', 'tham quan', 'Viện trưởng', 'A. Cựu', 'biểu tình', 'nhuốm', 'Vicente', 'Morocco', 'phương pháp', 'trồng trọt', 'tương bần', 'lộ thiên', 'trình diễn', 'bàng quang', 'giấc mộng', 'bé nhỏ', 'trôi', 'khúc tục', 'trend', 'áo phông', 'Trùng Khánh thành phố', 'Martinez', 'Hoàng gia', 'Hồ Đức Phớc', 'thân phận', 'Dugarry', 'thẩm mỹ', 'long biên', 'mê hoặc', 'United Technologies', 'Rice', 'né tránh', 'trường học', 'tàn bạo', 'xã hộiThách thức', 'Đầu cuộc', 'trọng đại', 'xen kẽ', 'tập kích', 'tru

In [28]:
vocab_list = list(vocab_set)
word_features = pd.DataFrame(np.zeros([len(combined_df), len(vocab_set)]), columns=vocab_list)
word_features['topic'] = combined_df['topic']

print(word_features.shape)

(582, 14540)


In [ ]:
for i in range(len(combined_df)):
  current_text_list = combined_df["text"][i]
  for word in vocab_set:
    word_features.loc[i, word] = current_text_list.lower().count(word.lower())

In [ ]:
word_features.shape

(609, 14834)

In [ ]:
display(word_features)

,Tâm Ngọc,cảnh báo,rộng rãi,Bách,Álvaro,diện hợp,ngành,giải tỏa,bản năng,flow,...,Mara,rộng mở,việc làmphát triển,chiến tranh Tháng,châu lục,Reuters,màng não,tỉnh táo,đối phương,topic
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Thể thao
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Thể thao
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Thể thao
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,Thể thao
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,Thể thao
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Quan hệ quốc tế
605,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Quan hệ quốc tế
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Quan hệ quốc tế
607,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Quan hệ quốc tế


In [ ]:
y = word_features['topic']
X = word_features.drop('topic', axis=1)

print(f"Shape of X (features): {X.shape}")
print(f"Shape of y (target): {y.shape}")

Shape of X (features): (609, 14833)
Shape of y (target): (609,)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
def TrainModel(model, _X_train, _X_test, _y_train, _y_test):
    model.fit(_X_train, _y_train)

    y_pred = model.predict(_X_test)

    acc = accuracy_score(_y_test, y_pred)
    prec = precision_score(_y_test, y_pred, average='macro', zero_division=0)
    rec = recall_score(_y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(_y_test, y_pred, average='macro', zero_division=0)
    print(f"Model {model} - Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")

    return model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

logistic = TrainModel(LogisticRegression(solver='liblinear', random_state=42), X_train, X_test, y_train, y_test)
svm = TrainModel(SVC(kernel="sigmoid"), X_train, X_test, y_train, y_test)
nb = TrainModel(MultinomialNB(), X_train, X_test, y_train, y_test)

Model LogisticRegression(random_state=42, solver='liblinear') - Accuracy: 0.6230, Precision: 0.6702, Recall: 0.6215, F1: 0.6028
Model SVC(kernel='sigmoid') - Accuracy: 0.5902, Precision: 0.5010, Recall: 0.4962, F1: 0.4700
Model MultinomialNB() - Accuracy: 0.6393, Precision: 0.6094, Recall: 0.6502, F1: 0.5780


**Test với vài mẫu dữ liệu:**

s1 = "Hai tuyến đường sắt đô thị nối miền đông nam bộ và miền tây nam bộ sẽ sớm được khởi công"

s2 = "Các nhà khoa học đã tìm ra phương pháp điều trị ung thư mới"

s3 = "Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha"

In [ ]:
def predict_news(s, model, vocab_):
    w_vi = " | ".join(underthesea.word_tokenize(s))
    vector_input = pd.DataFrame(np.zeros([1, len(vocab_)]), columns=vocab_)
    for word in vocab_:
        vector_input.loc[0, word] = w_vi.lower().count(word.lower())
    result = model.predict(vector_input)
    return {s: result[0]}


In [ ]:
input_list = ["Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.",
              "Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.",
              "Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha"]

In [ ]:
for i in range(len(input_list)):
  print("Logistic: ", predict_news(input_list[i], logistic, vocab_list))

Logistic:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
Logistic:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
Logistic:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Xã hội - Đời sống'}


In [ ]:
for i in range(len(input_list)):
  print("SVM: ", predict_news(input_list[i], svm, vocab_list))

SVM:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
SVM:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
SVM:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Xã hội - Đời sống'}


In [ ]:
for i in range(len(input_list)):
  print("NB: ", predict_news(input_list[i], nb, vocab_list))

NB:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Kinh doanh'}
NB:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'môi trường và thiên tai'}
NB:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Thể thao'}


***Câu 2: Loại bỏ các từ trùng lặp giữa các category***

In [ ]:
list_category = {}
for i in range(len(combined_df_by_category["topic"])):
    list_category[combined_df_by_category["topic"][i]] = set((combined_df_by_category["title"][i] + " | " + combined_df_by_category["content"][i]).split(" | "))

In [ ]:
intersected = set(list_category[list(list_category.keys())[0]])
for i in range(1, len(list_category)):
    intersected = intersected.intersection(list_category[list(list_category.keys())[i]])

In [ ]:
print(intersected)

{'khỏe', 'thêm', 'theo', 'xây dựng', 'nước', 'kinh tế', 'hàng', 'Việt', 'tiếng', 'tình trạng', 'tăng', 'nhiều', 'giữa', 'không', 'giảm', 'nhằm', 'phải', 'kéo dài', 'hoạt động', 'trên', 'theo dõi', 'tập trung', 'sang', 'buổi', 'giúp', 'có thể', 'trong', 'được', 'nỗ lực', 'Nhiều', 'đánh giá', 'phát triển', 'trường', 'đúng', 'mạng', 'thị trường', 'địa phương', 'việc', 'cách', 'đặc biệt', 'quản lý', 'hoặc', 'nhưng', 'vùng', 'cùng', 'ý kiến', 'cộng đồng', 'thời gian', 'ngày', 'hỗ trợ', 'sống', 'xã hội', 'giai đoạn'}


In [ ]:
vocab_set_2 = vocab_set.difference(intersected)

In [ ]:
print(len(vocab_set))
print(len(vocab_set_2))
vocab_list_2 = list(vocab_set_2)

14833
14780


In [ ]:
word_features2 = pd.DataFrame(np.zeros([len(combined_df), len(vocab_set_2)]), columns=vocab_list_2)
word_features2['topic'] = combined_df['topic']
print(word_features2.shape)

(609, 14781)


In [ ]:
diff_columns = word_features.columns.difference(word_features2.columns)
new_word_feature = word_features.drop(columns=diff_columns)

print(new_word_feature.shape)

(609, 14781)


In [ ]:
y2 = new_word_feature['topic']
X2 = new_word_feature.drop('topic', axis=1)

print(f"Shape of X (features): {X2.shape}")
print(f"Shape of y (target): {y2.shape}")

Shape of X (features): (609, 14780)
Shape of y (target): (609,)


In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.1, random_state=42)

In [ ]:
logistic2 = TrainModel(LogisticRegression(solver='liblinear', random_state=42), X_train2, X_test2, y_train2, y_test2)
svm2 = TrainModel(SVC(kernel="sigmoid"), X_train2, X_test2, y_train2, y_test2)
nb2 = TrainModel(MultinomialNB(), X_train2, X_test2, y_train2, y_test2)

Model LogisticRegression(random_state=42, solver='liblinear') - Accuracy: 0.6230, Precision: 0.6334, Recall: 0.6215, F1: 0.5911
Model SVC(kernel='sigmoid') - Accuracy: 0.5738, Precision: 0.4971, Recall: 0.4780, F1: 0.4594
Model MultinomialNB() - Accuracy: 0.6230, Precision: 0.5899, Recall: 0.6320, F1: 0.5547


In [ ]:
for i in range(len(input_list)):
  print("Logistic: ", predict_news(input_list[i], logistic2, vocab_list_2))

Logistic:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
Logistic:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
Logistic:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Xã hội - Đời sống'}


In [ ]:
for i in range(len(input_list)):
  print("SVM: ", predict_news(input_list[i], svm2, vocab_list_2))

SVM:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
SVM:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
SVM:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Xã hội - Đời sống'}


In [ ]:
for i in range(len(input_list)):
  print("NB: ", predict_news(input_list[i], nb2, vocab_list_2))

NB:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Kinh doanh'}
NB:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'môi trường và thiên tai'}
NB:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Thể thao'}


***Thử nghiệm chỉ sử dụng danh sách từ vựng là từ ghép (có ký tự trắng) hoặc tên riêng (viết hoa chữ cái đầu) hoặc tên viết tắt (viết hoa hoàn toàn)***

In [ ]:
def Feature2(s):
    return (" " in s) or s[0].isupper() or s.isupper()

In [ ]:
vocab_set_3 = set()
for w in vocab_set:
    if Feature2(w):
        vocab_set_3.add(w)

In [ ]:
print(len(vocab_set))
print(len(vocab_set_2))
print(len(vocab_set_3))
vocab_list_3 = list(vocab_set_3)

14833
14780
13402


In [ ]:
word_features3 = pd.DataFrame(np.zeros([len(combined_df), len(vocab_set_3)]), columns=vocab_list_3)
word_features3['topic'] = combined_df['topic']
print(word_features3.shape)

(609, 13403)


In [ ]:
diff_columns2 = word_features.columns.difference(word_features3.columns)
new_word_feature2 = word_features.drop(columns=diff_columns2)

print(new_word_feature2.shape)

(609, 13403)


In [ ]:
y3 = new_word_feature2['topic']
X3 = new_word_feature2.drop('topic', axis=1)

print(f"Shape of X (features): {X3.shape}")
print(f"Shape of y (target): {y3.shape}")

Shape of X (features): (609, 13402)
Shape of y (target): (609,)


In [ ]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.1, random_state=42)

In [ ]:
logistic3 = TrainModel(LogisticRegression(solver='liblinear', random_state=42), X_train3, X_test3, y_train3, y_test3)
svm3 = TrainModel(SVC(kernel="sigmoid"), X_train3, X_test3, y_train3, y_test3)
nb3 = TrainModel(MultinomialNB(), X_train3, X_test3, y_train3, y_test3)

Model LogisticRegression(random_state=42, solver='liblinear') - Accuracy: 0.6393, Precision: 0.6409, Recall: 0.6426, F1: 0.6067
Model SVC(kernel='sigmoid') - Accuracy: 0.5902, Precision: 0.5350, Recall: 0.5712, F1: 0.5153
Model MultinomialNB() - Accuracy: 0.6721, Precision: 0.6376, Recall: 0.6632, F1: 0.6004


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

for i in range(len(input_list)):
  print("Logistic: ", predict_news(input_list[i], logistic3, vocab_list_3))

Logistic:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
Logistic:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
Logistic:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Xã hội - Đời sống'}


In [ ]:
for i in range(len(input_list)):
  print("SVM: ", predict_news(input_list[i], svm3, vocab_list_3))

SVM:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
SVM:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
SVM:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Xã hội - Đời sống'}


In [ ]:
for i in range(len(input_list)):
  print("NB: ", predict_news(input_list[i], nb3, vocab_list_3))

NB:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
NB:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
NB:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Xã hội - Đời sống'}


***Câu 3. Sử dụng BoW**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline

corpus = combined_df['text'].astype(str).tolist()
corpus_norm = [w.replace(" ", "_").replace("_|_", " ") for w in corpus] # Thay đổi biểu diễn chuỗi "Bầu trời | trong xanh" thành "Bầu_trời trong_xanh"
y_cv = combined_df['topic']

X_train_text, X_test_text, y_train_cv, y_test_cv = train_test_split(corpus_norm, y_cv, test_size=0.1, random_state=42)

pipe_logistic = make_pipeline(CountVectorizer(), LogisticRegression(solver='liblinear', random_state=42))
pipe_svm = make_pipeline(CountVectorizer(), SVC(kernel='sigmoid'))
pipe_nb = make_pipeline(CountVectorizer(), MultinomialNB())

pipe_logistic.fit(X_train_text, y_train_cv)
pipe_svm.fit(X_train_text, y_train_cv)
pipe_nb.fit(X_train_text, y_train_cv)

def eval_pipeline(pipe, X_test, y_test):
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    print(f"Model {pipe.steps[-1][1].__class__.__name__} - Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")

eval_pipeline(pipe_logistic, X_test_text, y_test_cv)
eval_pipeline(pipe_svm, X_test_text, y_test_cv)
eval_pipeline(pipe_nb, X_test_text, y_test_cv)

vectorized_logistic = pipe_logistic
vectorized_svm = pipe_svm
vectorized_nb = pipe_nb

Model LogisticRegression - Accuracy: 0.7049, Precision: 0.7415, Recall: 0.6935, F1: 0.6929
Model SVC - Accuracy: 0.6393, Precision: 0.6441, Recall: 0.6131, F1: 0.5770
Model MultinomialNB - Accuracy: 0.6557, Precision: 0.5999, Recall: 0.6494, F1: 0.5688


In [ ]:
def predict_news_pipeline(s, pipe):
    w_vi = " | ".join(underthesea.word_tokenize(s)).replace(" ", "_").replace("_|_", " ")  # Thay đổi biểu diễn chuỗi "Bầu trời | trong xanh" thành "Bầu_trời trong_xanh"
    result = pipe.predict([w_vi])
    return {s: result[0]}

for s in input_list:
    print("Logistics: ", predict_news_pipeline(s, vectorized_logistic))

for s in input_list:
    print("SVM: ", predict_news_pipeline(s, vectorized_svm))

for s in input_list:
    print("NB: ", predict_news_pipeline(s, vectorized_nb))

Logistics:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
Logistics:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
Logistics:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Xã hội - Đời sống'}
SVM:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
SVM:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
SVM:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Xã hội - Đời sống'}
NB:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Kinh doanh'}
NB:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Sức khoẻ - y tế'}
NB:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Thể thao'}


***Câu 4. Sử dụng Tfidf***

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

tfidf_logistic = make_pipeline(TfidfVectorizer(), LogisticRegression(solver='liblinear', random_state=42))
tfidf_svm = make_pipeline(TfidfVectorizer(), SVC(kernel='sigmoid'))
tfidf_nb = make_pipeline(TfidfVectorizer(), MultinomialNB())

tfidf_logistic.fit(X_train_text, y_train_cv)
tfidf_svm.fit(X_train_text, y_train_cv)
tfidf_nb.fit(X_train_text, y_train_cv)

eval_pipeline(tfidf_logistic, X_test_text, y_test_cv)
eval_pipeline(tfidf_svm, X_test_text, y_test_cv)
eval_pipeline(tfidf_nb, X_test_text, y_test_cv)

Model LogisticRegression - Accuracy: 0.6230, Precision: 0.6546, Recall: 0.6053, F1: 0.5633
Model SVC - Accuracy: 0.7541, Precision: 0.8056, Recall: 0.7344, F1: 0.7085
Model MultinomialNB - Accuracy: 0.4918, Precision: 0.5318, Recall: 0.5008, F1: 0.4385


In [ ]:
for s in input_list:
    print("Logistic: ", predict_news_pipeline(s, tfidf_logistic))
for s in input_list:
    print("SVM: ", predict_news_pipeline(s, tfidf_svm))
for s in input_list:
    print("NB: ", predict_news_pipeline(s, tfidf_nb))

Logistic:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
Logistic:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
Logistic:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Xã hội - Đời sống'}
SVM:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
SVM:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
SVM:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Thể thao'}
NB:  {'Sau khi tổng thống Trump đắc cử, ông đã ra một loạt quyết sách về thuế.': 'Xã hội - Đời sống'}
NB:  {'Các nhà nghiên cứu của viện MIT đã tìm ra phương pháp điều trị ung thư mới dựa trên liệu pháp gene.': 'Xã hội - Đời sống'}
NB:  {'Siêu sao Christian Ronaldo là cầu thủ bóng đá quốc tịch Bồ đào nha': 'Thể thao'}
